# Import Module

In [23]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd

# Load Data

In [24]:
temps = pd.read_csv("./weather.csv")
temps.head()

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森


# Scraping

In [31]:
# dfs = pd.DataFrame()

In [32]:
path = "/home/higuchi-lab/anaconda3/lib"\
       "/python3.8/site-packages/chromedriver_binary/chromedriver"
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/"\
      "daily_s1.php?prec_no=44&block_no=47662&year=2022&month=05&day=&view=p1s"
# TODO: should change URL automatically correspoding to temps["area"].unique()

browser = webdriver.Chrome(executable_path=path)
browser.get(url)

In [33]:
target = temps["area"].unique()[31]
print(target)

東京


In [34]:
df = pd.DataFrame()

for i in range(31):
    where = 5 + i
    texts = browser.find_element_by_xpath(
        f"/html/body/div[2]/div/div[2]/div[2]/div[4]/table/tbody/tr[{where}]"
    )
    texts = texts.text.split(" ")
    texts = [text for text in texts if text != ")"]
    row = {
        "date": f"2022-05-{i+1}",
        "mean_temp": texts[5],
        "max_temp": texts[6],
        'max_temp_time': 0,
        "min_temp": texts[7],
        "min_temp_time": 0,
        "sum_rain": texts[2],
        "sun_time": texts[15],
        "mean_humid": texts[8],
        "area": target
    }
    df = pd.concat([df, pd.DataFrame(row, index=[i])], axis=0)

In [35]:
dfs = pd.concat([dfs, df], axis=0)

In [36]:
dfs

,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,2022-05-1,12.7,16.8,0,10.8,0,12.0,0.0,83,東京
1,2022-05-2,13.3,19.4,0,9.5,0,1.0,4.1,74,東京
2,2022-05-3,15.2,20.4,0,9.3,0,--,12.3,63,東京
3,2022-05-4,17.9,25.0,0,10.8,0,--,12.4,60,東京
4,2022-05-5,19.8,26.3,0,13.6,0,--,11.8,65,東京
5,2022-05-6,19.5,24.5,0,15.2,0,--,7.9,71,東京
6,2022-05-7,18.5,22.8,0,15.4,0,7.5,0.8,87,東京
7,2022-05-8,18.3,22.5,0,16.2,0,0.0,5.1,59,東京
8,2022-05-9,14.1,17.3,0,11.2,0,8.0,0.0,80,東京
9,2022-05-10,16.5,23.0,0,11.2,0,0.5,11.4,64,東京


In [37]:
# dfs.to_csv("./tmp_scraped_weather.csv", index=False)

# Concatenation Between Train Weather And Test Weather

In [38]:
temps = pd.read_csv("./weather.csv")
temps["date"] = pd.to_datetime(temps["date"], format="%Y%m%d")

test_temps = pd.read_csv("./tmp_scraped_weather.csv")
test_temps["date"] = pd.to_datetime(test_temps["date"], format="%Y-%m-%d")

temps = pd.concat([temps, test_temps], axis=0)
# temps.to_csv("./scraped_weather.csv", index=False)